Include some system headers:

In [1]:
#include <iostream>
#include <string>
#include <vector>

Read environment variables:

In [2]:
const std::string emotiEffLibRootDir(std::getenv("EMOTIEFFLIB_ROOT"));
const std::string emotiEffLibBuildDir(std::getenv("EMOTIEFFLIB_BUILD_DIR"));

std::cout << "EMOTIEFFLIB_ROOT: " << emotiEffLibRootDir << std::endl;
std::cout << "EMOTIEFFLIB_BUILD_DIR: " << emotiEffLibBuildDir << std::endl;

EMOTIEFFLIB_ROOT: /home/echuraev/Workspace/HSE/face-emotion-recognition
EMOTIEFFLIB_BUILD_DIR: /home/echuraev/Workspace/HSE/face-emotion-recognition/emotieffcpplib/build


Add linking with necessary libraries:

In [3]:
// NOTE: It is important that all paths in pragma should be specified as a string constant.
// Please, copy it from the output of the previous cell.

// Path to EmotiEffCpp lib dependencies
#pragma cling add_include_path("/home/echuraev/Workspace/HSE/face-emotion-recognition/emotieffcpplib/include")
#pragma cling add_include_path("/home/echuraev/Workspace/HSE/face-emotion-recognition/emotieffcpplib/3rdparty/opencv-mtcnn/lib/include")
#pragma cling add_include_path("/home/echuraev/Workspace/HSE/face-emotion-recognition/emotieffcpplib/3rdparty/xtensor/include")
// Path to OpenCV
#pragma cling add_include_path("/usr/include/opencv4")
// Path to ONNXRuntime
#pragma cling add_include_path("/home/echuraev/Workspace/HSE/onnxruntime/include")
// Path to Libtorch
#pragma cling add_include_path("/home/echuraev/Workspace/HSE/libtorch/include")

#pragma cling add_library_path("/home/echuraev/Workspace/HSE/face-emotion-recognition/emotieffcpplib/build/lib")
#pragma cling load("emotiefflib")
#pragma cling load("mtcnn")
#pragma cling load("libopencv_shape")
#pragma cling load("libopencv_stitching")
#pragma cling load("libopencv_objdetect")
#pragma cling load("libopencv_superres")
#pragma cling load("libopencv_videostab")
#pragma cling load("libopencv_calib3d")
#pragma cling load("libopencv_features2d")
#pragma cling load("libopencv_highgui")
#pragma cling load("libopencv_videoio")
#pragma cling load("libopencv_imgcodecs")
#pragma cling load("libopencv_video")
#pragma cling load("libopencv_photo")
#pragma cling load("libopencv_ml")
#pragma cling load("libopencv_imgproc")
#pragma cling load("libopencv_dnn")
#pragma cling load("libopencv_viz")

Include necessary headers:

In [4]:
#include <emotiefflib/facial_analysis.h>
#include <mtcnn/detector.h>

#include <xtensor/xarray.hpp>
#include <xtensor/xio.hpp>
#include <xtensor/xmath.hpp>
#include <xtensor/xsort.hpp>

To improve performance, we downscale the input image:

In [5]:
cv::Mat downscaleImageToWidth(const cv::Mat& inputImage, int targetWidth) {
    // Get the original dimensions
    int originalWidth = inputImage.cols;
    int originalHeight = inputImage.rows;

    if (originalWidth < targetWidth)
        return inputImage;

    // Calculate the scaling factor
    double scaleFactor = static_cast<double>(targetWidth) / originalWidth;

    // Calculate the new height while maintaining the aspect ratio
    int targetHeight = static_cast<int>(originalHeight * scaleFactor);

    // Resize the image
    cv::Mat outputImage;
    cv::resize(inputImage, outputImage, cv::Size(targetWidth, targetHeight));

    return outputImage;
}

Function for faces recognition:

In [6]:
std::vector<cv::Mat> recognizeFaces(const cv::Mat& frame, int downscaleWidth) {
    auto dirWithModels = emotiEffLibRootDir + "/emotieffcpplib/3rdparty/opencv-mtcnn/data/models";
    ProposalNetwork::Config pConfig;
    pConfig.protoText = dirWithModels + "/det1.prototxt";
    pConfig.caffeModel = dirWithModels + "/det1.caffemodel";
    pConfig.threshold = 0.6f;
    RefineNetwork::Config rConfig;
    rConfig.protoText = dirWithModels + "/det2.prototxt";
    rConfig.caffeModel = dirWithModels + "/det2.caffemodel";
    rConfig.threshold = 0.7f;
    OutputNetwork::Config oConfig;
    oConfig.protoText = dirWithModels + "/det3.prototxt";
    oConfig.caffeModel = dirWithModels + "/det3.caffemodel";
    oConfig.threshold = 0.7f;
    MTCNNDetector detector(pConfig, rConfig, oConfig);
    auto scaledFrame = downscaleImageToWidth(frame, downscaleWidth);
    double downcastRatioW = static_cast<double>(frame.cols) / scaledFrame.cols;
    double downcastRatioH = static_cast<double>(frame.rows) / scaledFrame.rows;
    std::vector<Face> faces = detector.detect(scaledFrame, 20.f, 0.709f);
    std::vector<cv::Mat> cvFaces = {};
    cvFaces.reserve(faces.size());
    for (auto& face : faces) {
        face.bbox.x1 *= downcastRatioW;
        face.bbox.x2 *= downcastRatioW;
        face.bbox.y1 *= downcastRatioH;
        face.bbox.y2 *= downcastRatioH;
        cv::Rect roi(face.bbox.x1, face.bbox.y1, face.bbox.x2 - face.bbox.x1,
                     face.bbox.y2 - face.bbox.y1);
        cv::Mat f = frame(roi).clone();
        cvFaces.push_back(f);
    }
    return cvFaces;
}

In [7]:
// std::string img2base64(const cv::Mat& img) {
//     std::vector<uchar> buf;
//     cv::imencode(".jpg", img, buf);
//     auto *enc_msg = reinterpret_cast<unsigned char*>(buf.data());
//     std::string encoded = base64_encode(enc_msg, buf.size());
//     return encoded;
// }

Open test image:

In [8]:
auto imagePath = emotiEffLibRootDir + "/tests/test_images/20180720_174416.jpg";
cv::Mat frame = cv::imread(imagePath);
cv::Mat frameRgb;
cv::cvtColor(frame, frameRgb, cv::COLOR_BGR2RGB);
//cv::imshow("Image", frameRgb);

Recognize faces:

In [9]:
auto facialImages = recognizeFaces(frame, 500);
std::string modelPath = emotiEffLibRootDir + "/models/emotieffcpplib_prepared_models/enet_b0_8_best_afew.onnx";
std::string backend = "onnx";
auto fer = EmotiEffLib::EmotiEffLibRecognizer::createInstance(backend, modelPath);
for (auto& face : facialImages) {
    auto res = fer->predictEmotions(face, true);
    auto pred = xt::argmax(res.scores, 1);
    std::cout << res.labels[0] << " == " << fer->getEmotionClassById(pred[0]) << std::endl;
}

Anger == Anger
Happiness == Happiness
Happiness == Happiness
